In [142]:
%reset -f
%matplotlib qt

##################################################
## cN1 vs cFN for a SINGLE Mach
##
##################################################
## Author: J.M. Roldan
## Date: 12/03/2020
## License: GPL 3.0
## Discord: TazX [Z+1]#0405
## Status: Final
##################################################
    
import pandas as pd
import math
from sklearn import preprocessing
from sklearn import linear_model
import matplotlib.pyplot as plt
import seaborn as sns

def regression(df, X, y, order):
    
    ######################################################
    # Fit and predict with Multiple Polynomial Regression
    ######################################################

    #generate a model of polynomial features
    poly = preprocessing.PolynomialFeatures(degree=order)

    #transform the x data for proper fitting (for single variable type it returns,[1,x,x**2])
    X_poly = poly.fit_transform(X)

    #generate the regression object
    regr = linear_model.LinearRegression()

    #preform the actual regression
    regr.fit(X_poly, y)

    #some statistics
    r_sq = regr.score(X_poly,y)
    b0 = regr.intercept_

    #new DF and check with original table
    columns = ['Mach', 'cN1', 'cFN_orig', 'cFN_predict', 'error']
    new_df = pd.DataFrame(columns=columns)

    #prediction loop
    for index in range(len(df.index)):
        predict_ = poly.fit_transform(([[df['Mach'][index], df['cN1'][index]]]))
        predCFN = regr.predict(predict_)
        sq_error = abs(predCFN[0]-df['cFN100'][index])**2
        new_df = new_df.append({'Mach':df['Mach'][index],\
                                'cN1':df['cN1'][index],\
                                'cFN_orig':df['cFN100'][index],\
                                'cFN_predict':predCFN[0],\
                                'error':sq_error}, ignore_index = True)

    mse = math.sqrt(sum(new_df['error'])/(len(new_df.index)-2))
    r_sq*=100
    print("Training --- Order: %d   R2 score: %.5f %%   Std Error: %.5f" % (order, r_sq, mse))

    return poly, regr, new_df

def predict(df, poly, regr, mach):
    ##################################
    #Predicted Curves build up
    ##################################
    m = regr.coef_
    Mach = mach
    Alt = 0
    cN1 = 0

    columns = ['cN1', 'cFN', 'Mach']
    cFN_predict = pd.DataFrame(columns=columns)

    #for index_mach in range (0, 10):
    for cN1 in range (0,115,5):
        predict_ = poly.fit_transform(([[Mach, cN1]]))
        predCFN = regr.predict(predict_)                                                  
        cFN_predict = cFN_predict.append({'cN1':cN1,'cFN':predCFN[0],'Mach':Mach}, ignore_index = True) 
        #Mach+=0.1

    sns.lineplot(data=cFN_predict, x="cN1", y="cFN", markers=True, ax=ax[1])
    
    cFN_predict.to_csv('n1_to_fn.csv', index = False, header=True)
    
    return new_df
    
################
# Figure Set-up
################

a4_dims = (11.7, 8.27)
sns.set(font_scale=1.0)
sns.set_palette("rocket")
fig, ax =plt.subplots(1,2, figsize=a4_dims)
ax[0].set(title='Original vs. Predicted Data', xlabel='cN1 (%)', ylabel='cFN (lbf)')
ax[1].set(title='cN1 vs Thrust @ Mach', ylim=(0, 40000), xlabel='cN1 (%)', ylabel='cFN (lbf)')

################
# Data Init
################
train = 0
phase = 0
Zero = 1
order = 2
m = 0.5

# CSV read and tiddy-up
df = pd.read_csv("a320-251n.csv")
df = df.rename(columns = {'ALT.':'Alt', 'MACH':'Mach'}, inplace=False)

if (Zero == 1):
    df = df[df['cFN'] > 0]
    df = df[df['Mach'] < (m+0.1)]
    df = df[df['Mach'] > (m-0.1)]

if (phase == 1):
    #df = df[df['PHASE'] != 'TO']
    #df = df[df['PHASE'] != 'CLB']
    #df = df[df['PHASE'] != 'CRZ']
    df = df[df['PHASE'] != 'DSC']
    #df = df[df['PHASE'] != 'HLD']
    
df = df.reset_index(drop=True)

# Define Independent and Dependent Variables
X = df[['Mach', 'cN1']]
y = df['cFN']

if (train == 1):
    for i in range(1,12):
        regression(df, X, y, i)
else:
    poly, regr, new_df = regression(df, X, y, order)
    new_df = predict(df, poly, regr, m)
    
    #Plot results
    sns.set_theme(style="darkgrid")
    sns.scatterplot(data=new_df, x="cN1", y="cFN_orig", s=8, ax=ax[0])
    sns.scatterplot(data=new_df, x="cN1", y="cFN_predict", s=8, ax = ax[0])
    
#poly.get_feature_names(['Alt', 'Mach', 'FF'])
#sns.scatterplot(data=df, x="cFN", y="Mach")

Training --- Order: 2   R2 score: 99.58407 %   Std Error: 8825.69507


In [141]:
%reset -f
%matplotlib qt

######################################################
# cN1 vs cFN100 for a MULTIPLE Mach
######################################################
    
import pandas as pd
import math
from sklearn import preprocessing
from sklearn import linear_model
import matplotlib.pyplot as plt
import seaborn as sns

def regression(df, X, y, order):
    
    ######################################################
    # Fit and predict with Multiple Polynomial Regression
    ######################################################

    #generate a model of polynomial features
    poly = preprocessing.PolynomialFeatures(degree=order)

    #transform the x data for proper fitting (for single variable type it returns,[1,x,x**2])
    X_poly = poly.fit_transform(X)

    #generate the regression object
    regr = linear_model.LinearRegression()

    #preform the actual regression
    regr.fit(X_poly, y)

    #some statistics
    r_sq = regr.score(X_poly,y)
    b0 = regr.intercept_

    #new DF and check with original table
    columns = ['Mach', 'cN1', 'cFN_orig', 'cFN_predict', 'error']
    new_df = pd.DataFrame(columns=columns)

    #prediction loop
    for index in range(len(df.index)):
        predict_ = poly.fit_transform(([[df['Mach'][index], df['cN1'][index]]]))
        predCFN = regr.predict(predict_)
        sq_error = abs(predCFN[0]-df['cFN'][index])**2
        new_df = new_df.append({'Mach':df['Mach'][index],\
                                'cN1':df['cN1'][index],\
                                'cFN_orig':df['cFN'][index],\
                                'cFN_predict':predCFN[0],\
                                'error':sq_error}, ignore_index = True)

    mse = math.sqrt(sum(new_df['error'])/(len(new_df.index)-2))
    r_sq*=100
    print("Training --- Order: %d   R2 score: %.5f %%   Std Error: %.5f" % (order, r_sq, mse))

    return poly, regr, new_df

def predict(df, poly, regr):
    ##################################
    #Predicted Curves build up
    ##################################
    m = regr.coef_
    Mach = 0
    Alt = 0
    cN1 = 0

    columns = ['cN1', 'cFN', 'Mach']
    cFN_predict = pd.DataFrame(columns=columns)

    for index_mach in range (0, 10):
        for cN1 in range (0,115,5):
            predict_ = poly.fit_transform(([[Mach, cN1]]))
            predCFN = regr.predict(predict_)                                                  
            cFN_predict = cFN_predict.append({'cN1':cN1,'cFN':predCFN[0],'Mach':Mach}, ignore_index = True) 
        Mach+=0.1

    sns.lineplot(data=cFN_predict, x="cN1", y="cFN", hue="Mach", markers=True, ax=ax[1])
    
    #cFN_predict.to_csv('n1_to_fn.csv', index = False, header=True)
    
    return new_df
    
################
# Figure Set-up
################

a4_dims = (11.7, 8.27)
sns.set(font_scale=1.0)
sns.set_palette("Spectral")
fig, ax =plt.subplots(1,2, figsize=a4_dims)
ax[0].set(title='Original vs. Predicted Data', xlabel='cN1 (%)', ylabel='cFN (lbf)')
ax[1].set(title='cN1 vs Thrust @ Mach', ylim=(0, 40000), xlabel='cN1 (%)', ylabel='cFN (lbf)')

################
# Data Init
################
train = 0
phase = 1
Zero = 0
order = 3

# CSV read and tiddy-up
df = pd.read_csv("a320-251n.csv")
df = df.rename(columns = {'ALT.':'Alt', 'MACH':'Mach'}, inplace=False)

if (Zero == 1):
    df = df[df['PCFN'] > 99]

if (phase == 1):
    df = df[df['PHASE'] != 'TO']
    #df = df[df['PHASE'] != 'CLB']
    #df = df[df['PHASE'] != 'CRZ']
    #df = df[df['PHASE'] != 'DSC']
    #df = df[df['PHASE'] != 'HLD']
    
df = df.reset_index(drop=True)

# Define Independent and Dependent Variables
X = df[['Mach', 'cN1']]
y = df['cFN']

if (train == 1):
    for i in range(1,12):
        regression(df, X, y, i)
else:
    poly, regr, new_df = regression(df, X, y, order)
    new_df = predict(df, poly, regr)
    
    #Plot results
    sns.set_theme(style="darkgrid")
    sns.scatterplot(data=new_df, x="cN1", y="cFN_orig", s=8, ax=ax[0])
    sns.scatterplot(data=new_df, x="cN1", y="cFN_predict", s=8, ax = ax[0])
    
#poly.get_feature_names(['Alt', 'Mach', 'FF'])
#sns.scatterplot(data=df, x="cFN", y="Mach")

Training --- Order: 3   R2 score: 99.12347 %   Std Error: 371.88144


In [137]:
%reset -f
%matplotlib qt

######################################################
# cN1 vs cFN for ONE by ONE Mach - FINAL
######################################################
    
import pandas as pd
import math
from sklearn import preprocessing
from sklearn import linear_model
import matplotlib.pyplot as plt
import seaborn as sns

def regression(df, X, y, order):
    
    ######################################################
    # Fit and predict with Multiple Polynomial Regression
    ######################################################

    #generate a model of polynomial features
    poly = preprocessing.PolynomialFeatures(degree=order)

    #transform the x data for proper fitting (for single variable type it returns,[1,x,x**2])
    X_poly = poly.fit_transform(X)

    #generate the regression object
    regr = linear_model.LinearRegression()

    #preform the actual regression
    regr.fit(X_poly, y)

    #some statistics
    r_sq = regr.score(X_poly,y)
    b0 = regr.intercept_

    #new DF and check with original table
    columns = ['Mach', 'cN1', 'cFN_orig', 'cFN_predict', 'error']
    new_df = pd.DataFrame(columns=columns)

    #prediction loop
    for index in range(len(df.index)):
        predict_ = poly.fit_transform(([[df['Mach'][index], df['cN1'][index]]]))
        predCFN = regr.predict(predict_)
        sq_error = abs(predCFN[0]-df['cFN'][index])**2
        new_df = new_df.append({'Mach':df['Mach'][index],\
                                'cN1':df['cN1'][index],\
                                'cFN_orig':df['cFN'][index],\
                                'cFN_predict':predCFN[0],\
                                'error':sq_error}, ignore_index = True)

    mse = math.sqrt(sum(new_df['error'])/(len(new_df.index)-2))
    r_sq*=100
    print("Training --- Order: %d   R2 score: %.5f %%   Std Error: %.5f" % (order, r_sq, mse))

    return poly, regr, new_df

def predict(df, poly, regr, Mach):
    ##################################
    #Predicted Curves build up
    ##################################
    m = regr.coef_
    Alt = 0
    cN1 = 0

    columns = ['cN1', 'cFN', 'Mach']
    cFN_predict = pd.DataFrame(columns=columns)
    
    for cN1 in range (0,115,5):
        predict_ = poly.fit_transform(([[Mach, cN1]]))
        predCFN = regr.predict(predict_)                                                  
        cFN_predict = cFN_predict.append({'cN1':cN1,'cFN':predCFN[0],'Mach':Mach}, ignore_index = True) 

    sns.lineplot(data=cFN_predict, x="cN1", y="cFN", hue="Mach", markers=True, ax=ax[1])
    
    file = "n1_fn_" + str(Mach) + ".csv"
    cFN_predict.to_csv(file, index = False, header=True)
    
    return new_df
    
################
# Figure Set-up
################

a4_dims = (11.7, 8.27)
sns.set(font_scale=1.0)
sns.set_palette("Spectral")
fig, ax =plt.subplots(1,2, figsize=a4_dims)
ax[0].set(title='Original vs. Predicted Data', xlabel='cN1 (%)', ylabel='cFN (lbf)')
ax[1].set(title='cN1 vs Thrust @ Mach', ylim=(0, 28000), xlabel='cN1 (%)', ylabel='cFN (lbf)')

################
# Data Init
################
train = 0
order = 3
m_range = [0, 0.105, 0.203, 0.319,0.409,0.516,0.604,0.7,0.82]
Mach = 0.203

# CSV read and tiddy-up
df = pd.read_csv("a320-251n_N1_FN.csv")
df = df.rename(columns = {'ALT.':'Alt', 'MACH':'Mach'}, inplace=False)
    
df = df[df['Mach'] == Mach]
    
df = df.reset_index(drop=True)

# Define Independent and Dependent Variables
X = df[['Mach', 'cN1']]
y = df['cFN']

if (train == 1):
    for i in range(1,12):
        regression(df, X, y, i)
else:
    poly, regr, new_df = regression(df, X, y, order)
    new_df = predict(df, poly, regr, Mach)
    
    #Plot results
    sns.set_theme(style="darkgrid")
    sns.scatterplot(data=new_df, x="cN1", y="cFN_orig", s=8, ax=ax[0])
    sns.scatterplot(data=new_df, x="cN1", y="cFN_predict", s=8, ax = ax[0])
    

Training --- Order: 3   R2 score: 100.00000 %   Std Error: 0.00000


In [144]:
%reset -f
%matplotlib qt

######################################################
# cN1 vs cFN Characterization - FINAL
######################################################
    
import pandas as pd
import math
from sklearn import preprocessing
from sklearn import linear_model
import matplotlib.pyplot as plt
import seaborn as sns

a4_dims = (11.7, 8.27)
sns.set(font_scale=1.0)
sns.set_palette("Spectral")
fig, ax =plt.subplots(1,1, figsize=a4_dims)
ax.set(title='cN1 vs Thrust @ Mach', ylim=(0, 28000), xlabel='cN1 (%)', ylabel='cFN (lbf)')

m_range = [0, 0.105, 0.203, 0.319,0.409,0.516,0.604,0.7,0.82]
all_files = []

for f in m_range:
    file = "./Engine Analysis/n1_fn_" + str(f) + ".csv"
    all_files.append(file)
    
combined_csv_data = pd.concat([pd.read_csv(f) for f in all_files])

sns.lineplot(data=combined_csv_data, x="cN1", y="cFN", hue="Mach", markers=True, ax=ax)

<AxesSubplot:title={'center':'cN1 vs Thrust @ Mach'}, xlabel='cN1 (%)', ylabel='cFN (lbf)'>